In [ ]:
%load_ext autoreload
%autoreload 2
# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
import os
from django.db.models import Count, Q, Func, FloatField, Value, Window
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import numpy as np
from django_react.settings import load_gin_config
import gin
from backend.ml_model.preference_aggregation_featureless import FeaturelessMedianPreferenceAverageRegularizationAggregator, loss_fcn, loss_fcn_gradient_hessian, variable_index_layer_call
import string
import shortuuid  # noqa: E402
from backend.models import DjangoUser
from backend.rating_fields import VIDEO_FIELDS
from backend.ml_model.client_server.django_ml_featureless import DatabasePreferenceLearnerFeatureless
import tensorflow as tf
from annoying.functions import get_object_or_None
from tqdm import tqdm
load_gin_config('backend/ml_model/config/featureless_config.gin')
gin.enter_interactive_mode()

In [ ]:
def random_alphanumeric(length=10, alphabet=None):
    """UUID1 without -."""
    if alphabet is None:
        alphabet = string.ascii_lowercase + string.digits
    res = shortuuid.ShortUUID(alphabet=alphabet).random(length=length)
    return str(res).replace('-', '')

In [ ]:
# creating an expert
e_username = random_alphanumeric()
e_django = DjangoUser.objects.create_user(username=e_username)
e = UserPreferences.objects.create(user=e_django) # INPUT FROM API
e_ui = UserInformation.objects.create(user=e_django)
domain = f"@{random_alphanumeric()}.com"
EmailDomain.objects.create(domain=domain, status=EmailDomain.STATUS_ACCEPTED)
email = VerifiableEmail.objects.create(user=e_ui, email=f"test{domain}", is_verified=True)

In [ ]:
def array_sample(arr, n):
    """Sample from an array, support non-integer arrays."""
    return [arr[i] for i in np.random.choice(range(len(arr)), n, replace=False)]

In [ ]:
# creating videos, INPUT FROM API
v = Video.objects.create(video_id=random_alphanumeric() + "__v")
w = Video.objects.create(video_id=random_alphanumeric() + "__w")

In [ ]:
# creating other videos to rate
other_videos = Video.objects.bulk_create([Video(video_id=random_alphanumeric()) for _ in range(50)])
other_videos = [Video.objects.get(video_id=v.video_id) for v in other_videos]

In [ ]:
def random_rating(scale=1):
    """Return a random rating."""
    return {f: np.random.rand() * 100 * scale for f in VIDEO_FIELDS}

In [ ]:
# creating ratings for v
for wother in array_sample(other_videos, 1):
    r = random_rating()
    y = ExpertRating.objects.create(user=e, video_1=v, video_2=wother, **r)

In [ ]:
# creating ratings for w
for wother in array_sample(other_videos, 1):
    r = random_rating()
    y = ExpertRating.objects.create(user=e, video_1=w, video_2=wother, **r)

In [ ]:
# filling video rating data
VideoRating.objects.create(user=e, video=v, **random_rating(scale=0.001))
VideoRating.objects.create(user=e, video=w, **random_rating(scale=0.001))

In [ ]:
for k, val in random_rating(scale=0.001).items():
    setattr(v, k, val)
v.save()
for k, val in random_rating(scale=0.001).items():
    setattr(w, k, val)
w.save()

In [ ]:
# rating for v/w
ExpertRating.objects.create(user=e, video_1=v, video_2=w, **{f: 0 for f in VIDEO_FIELDS})

In [ ]:
# INPUT FROM API
new_rating_vw = random_rating()
# new expert rating, not saved yet
new_rating = ExpertRating(user=e, video_1=v, video_2=w, **new_rating_vw)

In [ ]:
assert str(gin.query_parameter('learner.cls')) == '@DatabasePreferenceLearnerFeatureless',\
    "Only support featureless learner for online updates"

In [ ]:
def Y_ev(e, v):
    """Videos rated with v together by e."""
    Y_ev = Video.objects.filter(Q(expertrating_video_1__video_2=v,
                              expertrating_video_1__user=e)|
                            Q(expertrating_video_2__video_1=v,
                              expertrating_video_2__user=e)).distinct()
    return Y_ev

In [ ]:
MAX_YEV = 2

In [ ]:
Yev_sampled = Y_ev(e, v).exclude(id=w.id).order_by('?')[:MAX_YEV]
Yew_sampled = Y_ev(e, w).exclude(id=v.id).order_by('?')[:MAX_YEV]

In [ ]:
w1 = Yev_sampled[0]
for k, val in random_rating(scale=0.01).items():
    setattr(w1, k, val)
w1.save()

In [ ]:
Yev_sampled, Yew_sampled

In [ ]:
# subset of all videos used
videos_subset = [v, w] + list(Yev_sampled) + list(Yew_sampled)
videos_subset_v = [v, w] + list(Yev_sampled)
videos_subset_w = [w, v] + list(Yew_sampled)

In [ ]:
v_rating = ExpertRating.objects.filter(Q(video_1=v, video_2__in=Yev_sampled) |
                                       Q(video_1__in=Yev_sampled, video_2=v))
w_rating = ExpertRating.objects.filter(Q(video_1=w, video_2__in=Yew_sampled) |
                                       Q(video_1__in=Yew_sampled, video_2=w))

In [ ]:
e_ratings = list(v_rating) + list(w_rating) + [new_rating]
e_ratings_v = list(v_rating) + [new_rating]
e_ratings_w = list(w_rating) + [new_rating]

In [ ]:
videos_subset_v

In [ ]:
videos_subset_w

In [ ]:
e_ratings_v

In [ ]:
gin.bind_parameter('FeaturelessMedianPreferenceAverageRegularizationAggregator.epochs', 1)

In [ ]:
learner = DatabasePreferenceLearnerFeatureless(
    load=False, save=False, user_queryset=[e],
    video_queryset=videos_subset,
    users_to_ratings={e.id: e_ratings})

In [ ]:
# ONLY V
learner = DatabasePreferenceLearnerFeatureless(
    load=False, save=False, user_queryset=[e],
    video_queryset=videos_subset_v,
    users_to_ratings={e.id: e_ratings_v})

In [ ]:
learner.aggregator.loss_fcn_kwargs(**learner.aggregator.hypers)

In [ ]:
learner.aggregator.loss_fcn

In [ ]:
learner.users_to_ratings

In [ ]:
# loading the dataset
learner.fit()

In [ ]:
learner.users

In [ ]:
learner.videos

In [ ]:
learner.users_to_ratings

In [ ]:
# gradient subset

In [ ]:
# loading data from the database

In [ ]:
VideoRating.objects.get(video=v, user=e).features_as_vector

# format: user, video, feature
idxes = []
values = []

for video in learner.aggregator.all_ratings.objects:
    video_id_internal = learner.aggregator.all_ratings.objects_reverse[video]
    for user in learner.users + ['__aggregate_expert__']:
        expert_id_internal = learner.aggregator.all_ratings.experts_reverse[user]
        
        r = None
        if user == '__aggregate_expert__':
            r = Video.objects.get(video_id=video).features_as_vector
        else:
            r = get_object_or_None(VideoRating, video__video_id=video, user=e)
            if r:
                r = r.features_as_vector
        
        if r is not None:
            for i, val in enumerate(r):
                if val is not None and not np.isnan(val):
                    idxes.append((expert_id_internal, video_id_internal, i))
                    values.append(val)

In [ ]:
if idxes:
    learner.aggregator.all_ratings.layer.v =\
        tf.Variable(tf.tensor_scatter_nd_update(\
        learner.aggregator.all_ratings.layer.v,\
        idxes, values), trainable=True)

In [ ]:
learner.all_ratings.experts_reverse, learner.all_ratings.objects_reverse

In [ ]:
video_ids_vw = [learner.aggregator.all_ratings.objects_reverse[z.video_id] for z in [v, w]]
vid_v_int, vid_w_int = video_ids_vw

In [ ]:
# TODO: disable in prod, as value might be non-existent
check_values = True
if check_values:
    assert np.allclose(learner.all_ratings.layer.v[1, vid_v_int, :], v.features_as_vector)
    assert np.allclose(learner.all_ratings.layer.v[1, vid_w_int, :], w.features_as_vector)

    assert np.allclose(learner.all_ratings.layer.v[0, vid_v_int, :],
                       VideoRating.objects.get(user=e, video=v).features_as_vector)
    assert np.allclose(learner.all_ratings.layer.v[0, vid_w_int, :],
                       VideoRating.objects.get(user=e, video=w).features_as_vector)

In [ ]:
learner.aggregator.all_ratings.layer.v

In [ ]:
mb = learner.aggregator.sample_minibatch()

In [ ]:
mb

In [ ]:
if not mb:
    print("Minibatch is not valid")

In [ ]:
r_var = learner.aggregator.all_ratings.model.variables[0]

In [ ]:
#del learner.aggregator.hypers['ignore_vals']# = ['model_tensor']

loss_fcn_kwargs = learner.aggregator.loss_fcn_kwargs(**learner.aggregator.hypers)

In [ ]:
loss_fcn(**mb, **loss_fcn_kwargs, model_tensor=r_var)

In [ ]:
model = learner.aggregator.all_ratings.model

In [ ]:
self = learner.aggregator

In [ ]:
# internal indices for experts and objects (ratings)
idx_v1 = tf.stack((mb['experts_rating'], mb['objects_rating_v1']), axis=1)
idx_v2 = tf.stack((mb['experts_rating'], mb['objects_rating_v2']), axis=1)

# 2D array (comparison_id, feature) -> float
theta_eqv = model(idx_v1)
theta_eqw = model(idx_v2)
theta_vw = theta_eqv - theta_eqw
# print(theta_vw.shape, cmp.shape)
theta_vw_y = tf.math.multiply(theta_vw, mb['cmp'])
phi_eqvw = tf.exp(theta_vw_y)
phi_eqvw_inv = 1 / phi_eqvw

# value for the current video
# model rating
theta_eqv_video = model(tf.constant([0, 0]))
# common rating
s_eqv_video = model(tf.constant([1, 0]))
# epsilon
eps_eqv = tf.sign(theta_eqv_video - s_eqv_video)
Yev_video = mb['num_ratings_all'][0]

In [ ]:
tau_ev = loss_fcn_kwargs['lambda_'] * Yev_video / (loss_fcn_kwargs['C'] + Yev_video)

In [ ]:
grad_1 = tau_ev * eps_eqv

In [ ]:
gamma_eqv = tf.reduce_sum(mb['cmp'] / (1 + phi_eqvw_inv), axis=0)
grad_2 = gamma_eqv

In [ ]:
grad = grad_1 + grad_2

In [ ]:
grad

In [ ]:
hess_diag = tf.reduce_sum(mb['cmp'] ** 2 * phi_eqvw_inv / (1 + phi_eqvw_inv) ** 2, axis=0)

In [ ]:
model.layers[1].v

# Automatic gradient computation (slow due to diagnoal hessian)

In [ ]:
# only learn scores for v, w for the user (not common)
learn_mask = np.zeros_like(learner.aggregator.all_ratings.layer.v.numpy())
learn_mask[0, video_ids_vw, :] = 1
learn_mask = tf.constant(learn_mask)

In [ ]:
#%timeit learner.aggregator.loss_fcn(**mb)

In [ ]:
loss_fcn

In [ ]:
loss_fcn_kwargs = learner.aggregator.loss_fcn_kwargs(**learner.aggregator.hypers)

In [ ]:
loss_fcn_kwargs

In [ ]:
loss_fcn(**mb, **loss_fcn_kwargs, model_tensor=r_var)

In [ ]:
learner.aggregator.hypers['ignore_vals'] = ['model_tensor']
loss_fcn_kwargs = learner.aggregator.loss_fcn_kwargs(**learner.aggregator.hypers)

In [ ]:
r_var = tf.Variable(tf.constant(np.random.randn(*r_var.numpy().shape), dtype=tf.float32))

In [ ]:
r_var = learner.aggregator.all_ratings.layer.v

In [ ]:
@tf.function(experimental_relax_shapes=True)
def loss_fcn_gradient_hessian_11(video_indices, **kwargs):
    """Compute the loss function, gradient and the Hessian."""
    variable = kwargs['model_tensor']
    loss = loss_fcn(**kwargs)['loss']
    g = tf.gradients(loss, variable)[0]
    g = tf.gather(g, axis=1, indices=video_indices)
    h = tf.hessians(loss, variable)[0]
    h = tf.gather(h, axis=1, indices=video_indices)
    h = tf.gather(h, axis=4, indices=video_indices)
    
    s = tf.size(h[0, 0, 0])
    h = tf.reshape(tf.linalg.diag_part(tf.reshape(h, (s, s))), h.shape[:3])
    
    return {'loss': loss, 'gradient': g, 'hessian_diag': h}

In [ ]:
lgh = loss_fcn_gradient_hessian_11(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs,
           video_indices=tf.constant(video_ids_vw))

In [ ]:
correct_hessian = lgh['hessian_diag']

In [ ]:
correct_hessian

In [ ]:
%timeit loss_fcn_gradient_hessian_11(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs, video_indices=tf.constant(video_ids_vw))

In [ ]:
import seaborn as sns

In [ ]:
lgh = loss_fcn_gradient_hessian(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs,
           video_indices=tf.constant(video_ids_vw))

In [ ]:
sns.heatmap(np.log(np.abs(lgh['hessian'].numpy().reshape(28, 28))))

In [ ]:
video_ids_vw

In [ ]:
import tensorflow_probability as tfp

In [ ]:
r_var

In [ ]:
model_tensor = r_var

In [ ]:
@tf.function
def aaaaa():
    
#     def loss123(model_tensor):
    # LOSS COMMON TO 0
    experts_common_to_1 = loss_fcn_kwargs['aggregate_index'] * tf.ones(
        shape=mb['objects_common_to_1'].shape, dtype=tf.int64)
    idx_common_to_1 = tf.stack(
        (experts_common_to_1, mb['objects_common_to_1']), axis=1)
    s_qv_common_to_1 = variable_index_layer_call(model_tensor, idx_common_to_1)

    sm1 = tf.math.square(s_qv_common_to_1 - loss_fcn_kwargs['default_score_value'])

    # print(idx_common_to_1, s_qv_common_to_1, sm1)

    cto1 = tf.reduce_sum(sm1) * loss_fcn_kwargs['mu']
#     return cto1
    
    loss = cto1#loss123(model_tensor)
    grad = tf.gradients(loss, model_tensor)[0]

    
    hess = tfp.math.diag_jacobian(xs=model_tensor, ys=grad)
    
    tf.print(grad)
    tf.print(hess)

In [ ]:
aaaaa()

In [ ]:
correct_hessian

In [ ]:
@tf.function(experimental_relax_shapes=True)
def loss_fcn_gradient_diag_hessian(video_indices, **kwargs):
    """Compute the loss function, gradient and the Hessian."""
    variable = kwargs['model_tensor']
    loss = loss_fcn(**kwargs)['loss']
    g = tf.gradients(loss, variable)[0]
#     g_gather = tf.gather(g[0], axis=1, indices=video_indices)
    _, h = tfp.math.diag_jacobian(ys=g, xs=variable, sample_shape=[1])
    h = h[0]
#     h = tf.gather(h, axis=1, indices=video_indices)
    return {'loss': loss, 'gradient': g, 'diag_hessian': h}


In [ ]:
r_var = tf.Variable(tf.constant(np.random.randn(*r_var.numpy().shape), dtype=tf.float32))

In [ ]:
lgh = loss_fcn_gradient_diag_hessian(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs, video_indices=tf.constant(video_ids_vw))
lgh

In [ ]:
%timeit loss_fcn_gradient_diag_hessian(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs, video_indices=tf.constant(video_ids_vw))

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
r_var

In [ ]:
loss_fcn(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs)

In [ ]:
r_var = learner.aggregator.all_ratings.layer.v

In [ ]:
r_var

In [ ]:
@tf.function(experimental_relax_shapes=True)
def loss_fcn_gradient_hessian_11(video_indices, **kwargs):
    """Compute the loss function, gradient and the Hessian."""
    variable = kwargs['model_tensor']
    loss = loss_fcn(**kwargs)['loss']
    g = tf.gradients(loss, variable)[0]
#     g = tf.gather(g, axis=1, indices=video_indices)
    h = tf.hessians(loss, variable)[0]
#     h = tf.gather(h, axis=1, indices=video_indices)
#     h = tf.gather(h, axis=4, indices=video_indices)
    
    s = tf.size(h[0, 0, 0])
    h = tf.reshape(tf.linalg.diag_part(tf.reshape(h, (s, s))), h.shape[:3])
    
    return {'loss': loss, 'gradient': g, 'diag_hessian': h}

In [ ]:
losses = []
grads = []

for _ in tqdm(range(500)):

    lgh = loss_fcn_gradient_hessian_11(model_tensor=r_var.value(), **mb, **loss_fcn_kwargs,
                                         video_indices=tf.constant(video_ids_vw))
    
#     print(lgh['hessian'].shape)
#     orig_shape = lgh['hessian'].numpy().shape[:3]
#     s = np.prod(orig_shape)
#     print(orig_shape, s)
    
#     hess_reshape = lgh['hessian'].numpy().reshape((s, s))
#     lgh['diag_hessian'] = tf.constant(np.diag(hess_reshape).reshape(orig_shape))

    losses.append(lgh['loss'].numpy())

    hess_lr = 1. / lgh['diag_hessian']
    hess_lr_nonan = tf.raw_ops.Select(condition=tf.math.is_finite(hess_lr),
                                      x=hess_lr, y=tf.zeros_like(hess_lr))

    newton_grad = 0.01 * lgh['gradient'] * hess_lr_nonan

    mask = np.zeros_like(newton_grad)
    mask[0, 0, :] = 1

    r_var = tf.Variable(tf.raw_ops.Select(condition=mask, x=r_var - newton_grad, y=r_var))

    plt.hist(hess_lr_nonan[0, 0, :].numpy().flatten())
    grads.append(np.linalg.norm(lgh['gradient'].numpy().flatten()))

In [ ]:
plt.plot(losses)

In [ ]:
plt.plot(grads)

In [ ]:
lgh['diag_hessian']

In [ ]:
hess_lr

In [ ]:
hess_lr_nonan

In [ ]:
newton_grad